# CONTROL DE IMPRESORAS XEROX
### PREREQUISITOS
Hay que tener instalados los paquetes que lueo de importan en el entorno virtual

Utiliza un asrhivo de entrada que se llama ***Impresoras.csv***, que tiene que estar ubicado en el directorio que se identifica en la línea ***os.chdir()***

En este primer paso, realiza una verificación del estado de las impresoras y toma su tiempo de latencia y guarda un archivo intermedio "PING Procesado.cvs.
Este es la entrada para la parte final

La CELDAS se ejecutan en forma secuencial

In [ ]:
# Importa Librerias y Paguetes utilizados
import time
import datetime
import pandas as pd
import os
import platform
import subprocess
import wmi
import requests
from bs4 import BeautifulSoup
import io
import re
from lxml import html, etree
# Para el Modelo B400
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions  as EC
from selenium.webdriver.common.by import By

### PASO 1 Verificar PING y estado en linea
Procesa el archivo de entrada y genera un archivo intermedio verificando que impresoras están en línea.

In [ ]:
# RUTINA INICIAL
#Rutina de Preparación
print ("Arrancamos con el Proceso")
print('plataforma de ejecuión: ', platform.system().lower())  # Verificar plataforma ejecución

# Carga del archivo de impresoras en una matriz (ADECUAR A LA MAQUINA DE EJECUCIÓN)
os.chdir('C:/Users/eettlin.ENERGIAER/Downloads')
os.getcwd()
filename = 'Impresoras.csv'

data = pd.read_csv(filename, sep=';', header=0)
print('Matriz Orignal' , data.shape)

#Agrego Nuevas Columnas con valores por defecto
salida=data.assign(PING=0)
salida=salida.assign(TTL=0)
salida=salida.assign(FECHA=0)
salida=salida.assign(CONTADOR=0)
salida=salida.assign(N_SERIE=0)
#print (salida.head(10))

#Rutina de Búseda de Datos x Impresora
print ("Arrancamos con la Iteración")

# Prueba si hay conexión en todas las impresoras
procesado = 0
print('{0} {1:%H:%M:%S} {2}'.format(procesado, datetime.datetime.now(),
                                    "________________________________________"))
# Creo nuevo objeto WMI (Esto ejecuta en la consola de Windows el comando PING)
c = wmi.WMI()

# Bucle para recorrer el archivo de entrada y verificar ON-LINE
for i in range(len(salida)):
        Impresora = salida.iloc[i]['OBLEA']
        servidorIP = salida.iloc[i]['IP']
        modelo = salida.iloc[i]['MODELO']
        #print(i,'Input Host ',Impresora, ' - ',servidorIP)
        if servidorIP != '0' :
            host = servidorIP                       
        else:
            host = Impresora 
        # Con este comando se dispara el comando PING
        #print(i,'\tLlamamos al ping con parámetro: ',host, 'para la impresora', Impresora)
        x = c.Win32_PingStatus(Address= host)
        for d in x:            
            salida.loc[i,'TTL']=d.ResponseTime
            salida.loc[i,'PING']=d.StatusCode 
            salida.loc[i,'FECHA']=datetime.datetime.now() 
            #salida.loc[i,'FECHA']=chequeo_contador(servidorIP, modelo)        
        procesado += 1
        #print(i,'Host ',host, ' - ',d.ProtocolAddress,' - ', d.StatusCode)    
# Grabo salida de archivo intermedio
salida.to_csv('PING procesado.csv', sep=';', decimal=',')

### PASO 2: Cargar en memoria las FUNCIONES  de Extracción
En esta celda se definen todas las futina de extracción agrupadas por tipo de impresora.
Estas funciones serán convocadas por la rutina final para la lectura de los contadores
Ejecutar antes de la celda 3

In [ ]:
#Defino las funciones a utilizar para la búsqueda del contador de acuerdo al modelo

def abrir_url(adress):
    res = 'Vacio'
    try:
        res = requests.get(adress,timeout=30)
    except requests.ConnectionError as e:
        print("¡¡UPS!! Error de conexión. Asegúrese de estar conectado a Internet. Detalles técnicos a continuación.\n")
        print(str(e))                
    except requests.Timeout as e:
        print("OOPS!! Error de Timeout ")
        print(str(e))        
    except requests.RequestException as e:
        print("OOPS!! Error General")
        print(str(e))        
    except KeyboardInterrupt:
        print("Alguien cerró el programa") 
    except ConnectionRefusedError:
        print("Conexión Rechazada")
        print(str(e))         
    return res

# Funciones Agrupadas por Tipo de Impresora
def xerox3000(host, modelo):
    website ='http://'+ host +'/sws/app/information/counters/counters.json'
    #print(website)         
    numeroSerie = '0'
    contador = 0
    result = abrir_url(website)    
    if result != 'Vacio':
        contenido = result.text.strip()
        contenido = contenido.replace("	", "")
        contenido = contenido.replace(",", "")
        contenido = contenido.replace("{", "ETIQUETA : DATO")
        contenido = contenido.replace("}", "")         
        #print(contenido)
        salida = pd.read_csv(io.StringIO(contenido), sep=":")
        #print (salida.head(4))
        numeroSerie = salida.iloc[0][1].replace(" ","")        
        numeroSerie = numeroSerie.replace('"' , '')        
        contador = salida.iloc[6][1]        
        print( host, ' -- es una ', modelo ,' RUTINA: xerox3000 : NS = ',numeroSerie,' COPIAS ',contador)
    return contador, numeroSerie

def xerox3200(host, modelo):
    website ='http://'+ host +'/sws/app/information/counters/counters.json'
    #print(website)         
    numeroSerie = '0'
    contador = 0
    result = abrir_url(website)    
    if result != 'Vacio':
        contenido = result.text.strip()
        contenido = contenido.replace("	", "")
        contenido = contenido.replace(",", "")
        contenido = contenido.replace("{", "ETIQUETA : DATO")
        contenido = contenido.replace("}", "")         
        #print(contenido)
        salida = pd.read_csv(io.StringIO(contenido), sep=":")
        #print (salida.head(4))
        numeroSerie = salida.iloc[0][1].replace(" ","")        
        numeroSerie = numeroSerie.replace('"' , '')        
        contador = salida.iloc[6][1]        
        print( host, ' -- es una ', modelo ,' RUTINA: xerox3200 : NS = ',numeroSerie,' COPIAS ',contador)
    return contador, numeroSerie

def xerox3600(host, modelo):
    numeroSerie = '0'
    contador = 0
    website ='http://'+ host +'/status/statgeneral.htm'        
    result = abrir_url(website)
    if result != 'Vacio':
        contenido = result.text
        soup = BeautifulSoup(contenido,'html.parser')
        contador = soup.find('td',class_= 'std_2').get_text()
        
        # Rutina Nro de Serie
        website ='http://'+ host +'/setting/setsys.htm'
        result = requests.get(website)
        contenido = result.text 
        
        primeraO = contenido.find("Serial Number:")        
        terceraO = contenido.find("std_2>",primeraO+16)
        cuartaO = contenido.find("</td>",terceraO+6)
        numeroSerie = contenido[terceraO+6:cuartaO]
        numeroSerie = numeroSerie.replace("	", "")
        
        print( host, ' -- es una ', modelo ,' RUTINA: xerox3600 : NS = ',numeroSerie,' COPIAS ',contador)        
    return contador, numeroSerie 

def xerox6500(host, modelo):
    numeroSerie = '0'
    contador = 0
    website ='http://'+ host +'/status/statgeneralx.htm'   
    result = abrir_url(website)
    if result != 'Vacio':
        contenido = result.text
        soup = BeautifulSoup(contenido,'html.parser') 
        #print(soup.prettify())       
        contador = soup.find('td',class_= 'std_2').get_text()        
        numeroSerie = 'en desarrollo' 
        print( host, ' -- es una ', modelo ,' RUTINA: xerox6500 : NS = ',numeroSerie,' COPIAS ',contador)           
    return contador, numeroSerie

def xerox7000(host, modelo):
    numeroSerie = '0'
    contador = 0
    website ='http://'+ host +'/counters/usage.php' 
    result = abrir_url(website)
    if result != 'Vacio':
        contenido = result.text
        soup = BeautifulSoup(contenido,'html.parser')
        #print(soup.prettify())
        contador = soup.find('td',class_= 'normalTextCenter').get_text()

            
        # Rutina Nro de Serie
        website ='http://'+ host +'/counters/billing_info.php'
        result = requests.get(website)
        contenido = result.text

        primeraO = contenido.find("serialNum")        
        terceraO = contenido.find("</b>",primeraO+25)
        cuartaO = contenido.find("</div>",terceraO+6)
        numeroSerie = contenido[terceraO+6:cuartaO]
        numeroSerie = numeroSerie.strip()
        
        print( host, ' -- es una ', modelo ,' RUTINA: xerox7000 : NS = ',numeroSerie,' COPIAS ',contador)            
    return contador, numeroSerie

def xerox8800(host, modelo):
    numeroSerie = '0'
    contador = 0
    # Modelo Intermedio, con software Antiguo reformado
    website ='http://'+ host +'/status.html'   
    result = abrir_url(website)
    if result != 'Vacio':
        contenido = result.text
        # Rutina Contador de Impresiones
        primeraO = contenido.find("Recuen")
        segundaO = contenido.find("Recuen",primeraO+5)
        terceraO = contenido.find("<td>",segundaO+5)
        cuartaO = contenido.find("</td>",terceraO+5)
        contador = contenido[terceraO+4:cuartaO]
        # Rutina Numero de Serie
        website ='http://'+ host +'/aboutprinter.html' 
        result = requests.get(website)
        contenido = result.text
        primeraO = contenido.find("serie de la impresora")        
        terceraO = contenido.find("<td>",primeraO+15)
        cuartaO = contenido.find("</td>",terceraO+5)
        numeroSerie = contenido[terceraO+4:cuartaO]
 
        print( host, ' -- es una ', modelo ,' RUTINA: xerox8800 : NS = ',numeroSerie,' COPIAS ',contador)           
    return contador, numeroSerie

def xerox8900(host, modelo):
    numeroSerie = '0'
    contador = 0
    website ='http://'+ host +'/counters/usage.php'
    result = abrir_url(website)
    if result != 'Vacio':
        contenido = result.text
        soup = BeautifulSoup(contenido,'html.parser')
        #print(soup.prettify())
        contador = soup.find('td',class_= 'normalTextCenter').get_text()
        numeroSerie = 'en desarrollo'
        print( host, ' -- es una ', modelo ,' RUTINA: xerox8900 : NS = ',numeroSerie,' COPIAS ',contador)            
    return contador, numeroSerie

def xeroxBX00(host, modelo):
    numeroSerie = 'en desarrollo'
    contador = 0
    # FALTA DESARROLLAR ( Complicado Pagina dinámica JS)
    # Aplica a modelos B400, B600 y B605
    # Seguramente SELENIUM o SCRAPY
    website ='http://'+ host +'/home/index.html#hashHome'

    #Opciones de Navegación
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    driver_path ='C:\\Users\\eettlin.ENERGIAER\\OneDrive - ENERSA\\PY\\chromedriver.exe'
    driver = webdriver.Chrome(driver_path, chrome_options=options)

    #Inicializamos Navegador
    driver.get(website)
    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='openBillingDetailsModalWindow']"))).click()
    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='ui-id-4']/span/span"))).click()

    texto_columnas = driver.find_element( By.XPATH ,"//*[@id='billingDetailsModalRoot']/div[1]")
    contenido = texto_columnas.text
    #print(contenido)
    #print('______________________________________________')
    posinicio = contenido.find("del dispositivo: ")        
    posfin = contenido.find("Contadores de factura")
    numeroSerie = contenido[posinicio+17 : posfin]
    numeroSerie = numeroSerie.strip()
    #print("Inicio = ", posinicio)
    #print("Fin = ", posfin)
    #print ("Nro Serie= ", numeroSerie)
    #print('______________________________________________')
    posinicio = contenido.find("Total de impresiones")        
    posfin = contenido.find("Impresiones en negro")
    contador = contenido[posinicio+20 : posfin]
    contador = contador.strip()
    #print("Inicio = ", posinicio)
    #print("Fin = ", posfin)
    #print ("Contador = ", contador)
    #print('______________________________________________')
    
    print( host, ' -- es una ', modelo ,' RUTINA: xeroxBX00 : NS = ',numeroSerie,' COPIAS ',contador)
    driver.quit()            
    return contador, numeroSerie

def xeroxC500(host, modelo):
    contador = 0
    numeroSerie = 'en desarrollo'
    # FALTA DESARROLLAR ( Complicado Pagina dinámica JS)
    # Aplica a modelos C500
    # Seguramente SELENIUM o SCRAPY
    website ='http://'+ host +'/home/index.html#hashHome'
    result = abrir_url(website)
    if result != 'Vacio':
        contenido = result.text    
        print( host, ' -- es una ', modelo ,' RUTINA: xeroxC500 : NS = ',numeroSerie,' COPIAS ',contador)            
    return contador, numeroSerie

# BUCLIE PRINCIPAL DE CONTROL
def chequeo_contador(host, modelo):    
    if modelo == '3260':
        contador, numeroSerie = xerox3200(host, modelo)
    elif modelo == '3320':
        contador, numeroSerie = xerox3000(host, modelo)
    elif modelo == '3610':        # IDEM 3615
        contador, numeroSerie = xerox3600(host, modelo)         
    elif modelo == '3615':        # FUNCIONA PERFECTO
        contador, numeroSerie = xerox3600(host, modelo)
    elif modelo == '6500':
        contador, numeroSerie = xerox6500(host, modelo)
    elif modelo == '6505':
        contador, numeroSerie = xerox6500(host, modelo)
    elif modelo == '6655':
        contador, numeroSerie = xerox7000(host, modelo)      
    elif modelo == '7225':
        contador, numeroSerie = xerox7000(host, modelo)         
    elif modelo == '7845':
        contador, numeroSerie = xerox7000(host, modelo) 
    elif modelo == '8900':
        contador, numeroSerie = xerox7000(host, modelo)     
    elif modelo == '8880':  # FALTA CORREGIR
        contador, numeroSerie = xerox8800(host, modelo)  
    elif modelo == 'B400':      # UTILIZAR SELENIUM + LENTO        
        contador, numeroSerie = xeroxBX00(host, modelo)     
    elif modelo == 'B405':
        contador, numeroSerie = xeroxBX00(host, modelo)         
    elif modelo == 'B615':
        contador, numeroSerie = xeroxBX00(host, modelo) 
    elif modelo == 'C500':
        contador, numeroSerie = xeroxBX00(host, modelo)          
    else:        
        contador = 0
        numeroSerie ='Modelo No Definido'
    #Al finalizar la función, Devuelvo los datos hallados.
    return contador, numeroSerie

print( "MODULOS DE LECTURA CARGADOS")


### PASO 3: Recuperar los contadores a partir del archivo intermedio

En caso de necesitarse se puede evitar que sea leida alguna impresora cambiando el estado de la columna PING = 0 para que sea leido. Cualquier otro número considera que estúvo fuera de linea y no será leida.

In [ ]:
# Bucle para RECUPERAR LOS CONTADORES

#Leo el Archivo PRE PROCESADO
os.chdir('C:/Users/eettlin.ENERGIAER/Downloads')  # ESPECIFICAR EL DIRECTORIO DE TRABAJO
os.getcwd()
filename = 'PING procesado.csv'
PINGdata = pd.read_csv(filename, sep=';', decimal=',' ,header=0)
print('Matriz Orignal' , PINGdata.shape)

#Elimino la primer columna ID Registro
#print (PINGdata.head(5))
PINGdata = PINGdata.drop(PINGdata.columns[[0]], axis='columns')
print (PINGdata.head(5))
procesado = 0
numeroSerie = 0


for i in range(len(PINGdata)):
        Impresora = PINGdata.iloc[i]['OBLEA']
        servidorIP = PINGdata.iloc[i]['IP']
        modelo = PINGdata.iloc[i]['MODELO']
        enlinea = PINGdata.loc[i,'PING'] 
        procesar = PINGdata.loc[i,'PROCESAR'] 
        #print(i,'Input Host ',Impresora, ' - ',servidorIP, ' MODELO ', modelo, 'enlinea', enlinea)

        if (enlinea == 0) and (procesar == 1):
            PINGdata.loc[i,'CONTADOR'], PINGdata.loc[i,'N_SERIE']=chequeo_contador(servidorIP, str(modelo)) 
            PINGdata.loc[i,'FECHA']=datetime.datetime.now() 
            print (Impresora, ' en linea - MODELO:',modelo,' contador =', PINGdata.loc[i,'CONTADOR'], "Nro serie: ", PINGdata.loc[i,'N_SERIE'],)
                   
        procesado += 1
        #print(i,'Host ',host, ' - ',d.ProtocolAddress,' - ', d.StatusCode)    

print ("Fin de Iteración")
print('{0} {1:%H:%M:%S} {2}'.format(procesado, datetime.datetime.now(),"________________________________________"))
print (PINGdata.head(10))

# Grabo salida de archivo
PINGdata.to_csv('CONTADOR procesado.csv', sep=';', decimal=',')
print ("PROGRAMA FINALIZADO !!")

### FINAL DEL PROCESO.
En el directorio de trabajo queda el archivo que se puede abrir en excel CONTADOR PROCESADO

##SANDBOX - Prueba Individual (BORRAR)

In [ ]:
# Modelo XEROX B400
from lxml import html, etree
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions  as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service  # Cambio 2

import time
import pandas as pd

#Opciones de Navegación
options = webdriver.ChromeOptions()
#options.add_argument('--start-maximized')  # CON VENTANA INTERACTIVA
options.add_argument('--headless')      # CON VENTANA MINIMIZADA
options.add_argument('--disable-gpu')   # CON VENTANA MINIMIZADA (Linkeada a la opción anterior)

options.add_argument('--disable-extensions')

#driver_path ='C:\\Users\\eettlin.ENERGIAER\\OneDrive - ENERSA\\PY\\chromedriver.exe'  # ORIGINAL
#driver = webdriver.Chrome(driver_path, chrome_options=options)                        # ORIGINAL

s = Service('C:\\Users\\eettlin.ENERGIAER\\OneDrive - ENERSA\\PY\\chromedriver.exe')  # Cambio 1
driver = webdriver.Chrome(service=s, options=options)                          # Cambio 1



#Inicializamos Navegador
contador = 0
numeroSerie = 0
modelo ='B400'

host = '128.74.13.2'  #P0131 - PARANA

# Rutina Contador
website ='http://'+ host +'/home/index.html#hashHome'


driver.get(website)
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='openBillingDetailsModalWindow']"))).click()
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='ui-id-4']/span/span"))).click()

texto_columnas = driver.find_element( By.XPATH ,"//*[@id='billingDetailsModalRoot']/div[1]")

contenido = texto_columnas.text
#print(contenido)
print('______________________________________________')

posinicio = contenido.find("del dispositivo: ")        
posfin = contenido.find("Contadores de factura")
numeroSerie = contenido[posinicio+17 : posfin]
numeroSerie = numeroSerie.strip()

print("Inicio = ", posinicio)
print("Fin = ", posfin)
print ("Nro Serie= ", numeroSerie)

print('______________________________________________')
posinicio = contenido.find("Total de impresiones")        
posfin = contenido.find("Impresiones en negro")
contador = contenido[posinicio+20 : posfin]
contador = contador.strip()

print("Inicio = ", posinicio)
print("Fin = ", posfin)
print ("Contador = ", contador)


print('______________________________________________')
print( host, ' -- es una ', modelo ,' RUTINA: xeroxB400 : NS = ',numeroSerie,' COPIAS ',contador)  
driver.close()     

In [ ]:
# RUTINA 7000  # Impresoras COLOR
#host = '1128.0.13.23' #P0200  
#host = '128.0.13.27' #P0018 - 109249
host = '128.80.13.22' #P0018 - 109249

# def xerox7000(host, modelo):  # RUTINA IMPRESORAS COLOR
numeroSerie = '0'
contador = 0
contadorByN = 0
contadorCOL = 0

website ='http://'+ host +'/counters/usage.php' 
#result = abrir_url(website)
result = requests.get(website)

if result != 'Vacio':
    contenido = result.text
    soup = BeautifulSoup(contenido,'html.parser')
   #print(soup.prettify())
    table = soup.find('div',class_= 'boxBody tableBody')
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)

df = pd.DataFrame(res, columns=["Contador", "Total"])
print(df)
contador = df.iloc[0][1] 
contadorByN = df.iloc[1][1] 
contadorCOL = df.iloc[4][1] 

# Rutina Nro de Serie
website ='http://'+ host +'/counters/billing_info.php'
result = requests.get(website)
contenido = result.text

primeraO = contenido.find("serialNum")        
terceraO = contenido.find("</b>",primeraO+25)
cuartaO = contenido.find("</div>",terceraO+6)
numeroSerie = contenido[terceraO+6:cuartaO]
numeroSerie = numeroSerie.strip()

print( host, ' -- es una ', modelo ,' RUTINA: xerox7000 : NS = ',numeroSerie)
print (' COPIAS Totales',contador)
print (' COPIAS ByN',contadorByN)            
print (' COPIAS Coor',contadorCOL)            
#return contador,contadorByN, contadorCOL, numeroSerie

In [ ]:
### RUTINA (8900)

## COLOR QUBE 8900
#host = '128.0.13.107' #P0151 - 109249
host = '128.0.13.106' #P0151 - 109249

numeroSerie = '0'
contador = 0
contadorByN = 0
contadorCOL = 0
modelo ='8900'

website ='http://'+ host +'/counters/usage.php' 
#result = abrir_url(website)
result = requests.get(website)

if result != 'Vacio':
    contenido = result.text
    soup = BeautifulSoup(contenido,'html.parser')
   #print(soup.prettify())
    table = soup.find('div',class_= 'boxBody tableBody')
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)

df = pd.DataFrame(res, columns=["Contador", "Total"])
#print(df)
# CONTADOR COLOR Se calcula por Diferencia
contador = df.iloc[0][1] 
contadorByN = int(df.iloc[7][1]) + int(df.iloc[9][1])  + int(df.iloc[11][1]) + int(df.iloc[13][1])
contadorCOL = int(contador) - int(contadorByN)

# Rutina Nro de Serie
website ='http://'+ host +'/counters/billing_info.php'
result = requests.get(website)
contenido = result.text

primeraO = contenido.find("serialNum")        
terceraO = contenido.find("</b>",primeraO+25)
cuartaO = contenido.find("</div>",terceraO+6)
numeroSerie = contenido[terceraO+6:cuartaO]
numeroSerie = numeroSerie.strip()

print( host, ' -- es una ', modelo ,' RUTINA: xerox7000 : NS = ',numeroSerie)
print (' COPIAS Totales',contador)
print (' COPIAS ByN',contadorByN)            
print (' COPIAS Color',contadorCOL)            
#return contador,contadorByN, contadorCOL, numeroSerie

In [ ]:
# ARREGLEMOS LA B400N que pide ADMIN

host = '128.0.13.79'  #P0011 - 

# Modelo XEROX B400N
from lxml import html, etree
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions  as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service  # Cambio 2

import time
import pandas as pd

#Opciones de Navegación
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')  # CON VENTANA INTERACTIVA
#options.add_argument('--headless')      # CON VENTANA MINIMIZADA
#options.add_argument('--disable-gpu')   # CON VENTANA MINIMIZADA (Linkeada a la opción anterior)
options.add_argument('--disable-extensions')

#driver_path ='C:\\Users\\eettlin.ENERGIAER\\OneDrive - ENERSA\\PY\\chromedriver.exe'  # ORIGINAL
#driver = webdriver.Chrome(driver_path, chrome_options=options)                        # ORIGINAL

s = Service('C:\\Users\\eettlin.ENERGIAER\\OneDrive - ENERSA\\PY\\chromedriver.exe')  # Cambio 1
driver = webdriver.Chrome(service=s, options=options)   
                       # Cambio 1

#Inicializamos Navegador
contador = 0
numeroSerie = 0
modelo ='B400N'

# Rutina Contador
website ='http://'+ host +'/home/index.html#hashHome'
driver.get(website)

# Error Pagina No Segura
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='details-button']"))).click()
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='proceed-link']"))).click()

# Supero Ventana ERROR
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='notProcessedConfirm']"))).click()

# Ingreso USUARIO ADMIN
input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='loginName']")))
input_password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='loginPsw']")))
input_user.send_keys("admin")
input_password.send_keys("1111")
WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='loginButton']"))).click()

time.sleep(10)
# Navego hasta la página de Datos
WebDriverWait(driver,40).until(EC.element_to_be_clickable((By.XPATH, "//button[@id='openBillingDetailsModalWindow']"))).click()

WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='ui-id-4']/span/span"))).click()
# //*[@id="billingMeterUsageTabbedSet"]/div[1]/ul/li[2]   -  //*[@id="ui-id-7"]

texto_columnas = driver.find_element( By.XPATH ,"//*[@id='billingDetailsModalRoot']/div[1]")

contenido = texto_columnas.text
#print(contenido)
print('______________________________________________')

posinicio = contenido.find("del dispositivo: ")        
posfin = contenido.find("Contadores de factura")
numeroSerie = contenido[posinicio+17 : posfin]
numeroSerie = numeroSerie.strip()

#print("Inicio = ", posinicio)
#print("Fin = ", posfin)
print ("Nro Serie= ", numeroSerie)

print('______________________________________________')
posinicio = contenido.find("Total de impresiones")        
posfin = contenido.find("Impresiones en negro")
contador = contenido[posinicio+20 : posfin]
contador = contador.strip()

#print("Inicio = ", posinicio)
#print("Fin = ", posfin)
print ("Contador = ", contador)


print('______________________________________________')
print( host, ' -- es una ', modelo ,' RUTINA: xeroxB400 : NS = ',numeroSerie,' COPIAS ',contador)  
#driver.close()     

In [10]:
# NUEVO MODELO 7025 - En principio se parece a la C500 Pero no tanto

host = '128.0.13.175'  #Nueva COD en RACEDO
host = '128.0.13.173'  #Nueva COD en RACEDO

# http://128.0.13.175/home/index.html#hashHome

# ELIMINAR DE LA PRINCIPAL ESTAS IMPORTACIONES
from lxml import html, etree
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions  as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service  # Cambio 2

import time
import pandas as pd

numeroSerie = 'en desarrollo'
modelo = '7025'
contador = 0
contadorCOL = 0
contadorByN = 0

# Aplica a modelos  C500 y 7025 
website ='http://'+ host +'/home/index.html#hashHome'

#Opciones de Navegación
options = webdriver.ChromeOptions()
#options.add_argument('--start-maximized')  # CON VENTANA INTERACTIVA

options.add_argument('--headless')      # CON VENTANA MINIMIZADA
options.add_argument('--disable-gpu')   # CON VENTANA MINIMIZADA (Linkeada a la opción anterior)
options.add_argument('--disable-extensions')

s = Service('C:\\Users\\eettlin.ENERGIAER\\OneDrive - ENERSA\\PY\\chromedriver.exe')
driver = webdriver.Chrome(service=s, options=options)

#Inicializamos Navegador
driver.get(website)
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='openBillingDetailsModalWindow']"))).click()
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='ui-id-4']/span/span"))).click()

texto_columnas = driver.find_element( By.XPATH ,"//*[@id='billingDetailsModalRoot']/div[1]")
contenido = texto_columnas.text
print(contenido)
print('______________________________________________')
posinicio = contenido.find("del dispositivo: ")        
posfin = contenido.find("Contadores de factura")
numeroSerie = contenido[posinicio+17 : posfin]
numeroSerie = numeroSerie.strip()
#print("Inicio = ", posinicio)
#print("Fin = ", posfin)
#print ("Nro Serie= ", numeroSerie)
#print('______________________________________________')
posinicio = contenido.find("Total de impresiones")        
posfin = contenido.find("Impresiones en negro")
contador = contenido[posinicio+20 : posfin]
contador = contador.strip()
#print('______________________________________________')
posinicio = contenido.find("Impresiones en negro")        
posfin = contenido.find("Impresiones de tr. de copia en negro")
contadorByN = contenido[posinicio+20 : posfin]
contadorByN = contadorByN.strip()
#print('______________________________________________')
posinicio = contenido.find("Impresiones en color")        
posfin = contenido.find("Impresiones de tr. de copia en color")
contadorCOL = contenido[posinicio+20 : posfin]
contadorCOL = contadorCOL.strip()
print('______________________________________________')
print("Inicio = ", posinicio)
print("Fin = ", posfin)


print( host, ' -- es una ', modelo ,' RUTINA: xeroxC500 : NS = ',numeroSerie,' TOTAL ',contador,' BYN ',contadorByN,' COLOR ',contadorCOL)
driver.quit()            
# return contador,contadorByN, contadorCOL, numeroSerie

Número serie del dispositivo: 3396178159
Contadores de facturación
Contadores de uso
Total de impresiones 5812
Impresiones en negro 2345
Impresiones de tr. de impr. en negro 2345
Impresiones en color 3467
Impresiones de tr. de impr. en color 3467
Impresiones individuales 5812
Impresiones individuales en negro 2345
Impresiones individuales en color 3467
Impresiones de imágenes guardadas 0
Impresiones en negro de imágenes guar. 0
Impresiones en color de imág. guardadas 0
Hojas en blanco 1537
Hojas impresas en negro 1537
Hojas a dos caras impresas en blanco y negro 687
Hojas en color 3197
Hojas impresas en color 3197
Hojas a dos caras impresas en color 391
Hojas 4734
Hojas impresas 4734
Hojas a 2 caras 1078
Hojas impresas a 2 caras 1078
______________________________________________
______________________________________________
Inicio =  179
Fin =  -1
128.0.13.173  -- es una  7025  RUTINA: xeroxC500 : NS =  3396178159  TOTAL  5812  BYN  2345
Impresiones de tr. de impr. en negro 2345
Impr